                                      AI VIET NAM – AI COURSE 2024

                                Exercise: Multilayer Perceptrons (MLPs) and

                                         Activation Functions

                                Dinh-Thang Duong, Yen-Linh Vu, Quang-Vinh Dinh

                                        Ngày 12 tháng 11 năm 2024

In [56]:
!pip install torch-summary

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary


In [2]:
random_state = 59
np.random.seed(random_state)
if torch.cuda.is_available():
    torch.cude.manual_seed(random_state)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
data_path = '/kaggle/input/aio-datasets/NonLinear_data.npy'
dataset = np.load(data_path, allow_pickle=True).item()

In [48]:
X = dataset['X']
y = dataset['labels']
print(X.shape, y.shape)

(300, 2) (300,)


In [49]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      shuffle=True,
                                                      random_state=random_state,
                                                     )
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                      test_size=0.125, 
                                                      shuffle=True,
                                                      random_state=random_state,
                                                    )

In [50]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_valid = normalizer.transform(X_valid)
X_test = normalizer.transform(X_test)

In [51]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_valid = torch.tensor(X_valid, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [52]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [53]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_valid, y_valid)
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,                         
                         )
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                       )

In [54]:
class SoftmaxRegression(nn.Module):
    
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.output = nn.Linear(input_dims, output_dims)
        
    def forward(self, x):
        output = self.output(x)
        return output.squeeze(1)

In [80]:
class MLP_sigmoid(nn.Module):
    
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)
        output = self.output(x)
        return output.squeeze(1)

In [95]:
class MLP_tanh(nn.Module):
    
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.tanh(x)
        output = self.output(x)
        return output.squeeze(1)

In [102]:
input_dims = X_train.shape[1]
output_dims = torch.unique(y_train).shape[0]
hidden_dims = 512

# model = MLP_sigmoid(input_dims, hidden_dims, output_dims).to(device)
model = MLP_tanh(input_dims, hidden_dims, output_dims).to(device)


In [103]:
summary(model, (input_dims,))

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 512]                 1,536
├─Tanh: 1-2                              [-1, 512]                 --
├─Linear: 1-3                            [-1, 3]                   1,539
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.02


Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 512]                 1,536
├─Tanh: 1-2                              [-1, 512]                 --
├─Linear: 1-3                            [-1, 3]                   1,539
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.02

In [104]:
def compute_accuracy(y_true, y_pred):
    _, max_index = torch.max(y_pred, dim=1)
    num_correct = torch.sum(y_true == max_index).item()
    return num_correct / y_true.shape[0]

In [105]:
n_epochs = 100
lr = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [106]:
train_losses = []
val_losses = []
train_metrics = []
val_metrics = []

In [107]:
for ep in range(n_epochs):
    train_loss = 0.0
    train_y_pred = []
    train_y_true = []
    val_y_pred = []
    val_y_true = []
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_y_pred.append(output)
        train_y_true.append(y_batch)
    train_loss = train_loss / len(train_loader)
    train_y_true = torch.cat(train_y_true)
    train_y_pred = torch.cat(train_y_pred)
    train_metric = compute_accuracy(train_y_true, train_y_pred)
    train_losses.append(train_loss)
    train_metrics.append(train_metric)
    
    # evaluation:
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            output = model(X_batch)
            loss = criterion(output, y_batch)
            val_loss += loss.item()
            val_y_pred.append(output)
            val_y_true.append(y_batch)
        val_loss = val_loss / len(val_loader)
        val_y_pred = torch.cat(val_y_pred)  
        val_y_true = torch.cat(val_y_true)
        val_metric = compute_accuracy(val_y_true, val_y_pred)
    val_losses.append(val_loss)
    val_metrics.append(val_metric)
    
    
    print(f'EPOCH {ep + 1}: train_loss: {train_loss:.3f} || train_acc: {train_metric:.3f} || valid_loss: {val_loss:.3f} || valid_acc: {val_metric:.3f}')

EPOCH 1: train_loss: 0.861 || train_acc: 0.557 || valid_loss: 0.977 || valid_acc: 0.517
EPOCH 2: train_loss: 0.884 || train_acc: 0.562 || valid_loss: 0.915 || valid_acc: 0.533
EPOCH 3: train_loss: 0.759 || train_acc: 0.595 || valid_loss: 1.034 || valid_acc: 0.467
EPOCH 4: train_loss: 0.867 || train_acc: 0.571 || valid_loss: 0.908 || valid_acc: 0.433
EPOCH 5: train_loss: 0.791 || train_acc: 0.595 || valid_loss: 1.139 || valid_acc: 0.467
EPOCH 6: train_loss: 0.841 || train_acc: 0.571 || valid_loss: 0.865 || valid_acc: 0.467
EPOCH 7: train_loss: 0.760 || train_acc: 0.614 || valid_loss: 0.836 || valid_acc: 0.583
EPOCH 8: train_loss: 0.750 || train_acc: 0.643 || valid_loss: 1.061 || valid_acc: 0.617
EPOCH 9: train_loss: 0.769 || train_acc: 0.624 || valid_loss: 1.128 || valid_acc: 0.567
EPOCH 10: train_loss: 0.772 || train_acc: 0.629 || valid_loss: 0.824 || valid_acc: 0.533
EPOCH 11: train_loss: 0.808 || train_acc: 0.624 || valid_loss: 0.773 || valid_acc: 0.533
EPOCH 12: train_loss: 0.657 ||

In [108]:
model.eval()
with torch.no_grad():
    y_hat = model(X_test)
    test_acc = compute_accuracy(y_test, y_hat)
    print('Evaluation on validation set:')
    print(f'R2: {test_acc}')

Evaluation on validation set:
R2: 0.8666666666666667
